In [1]:

import mysql.connector as msql
import csv
import pandas as pd
from bulk_update.helper import bulk_update

user = 'minh'
# Connect to the PostgreSQL database
conn = msql.connect(
   database='comparator',
   user=user,
   password='07970797',
   host='database.cdjhn3xzznk3.eu-north-1.rds.amazonaws.com',
   port='3306'
)
print('connect succes')

# Create a cursor object
conn.autocommit = True
cursor = conn.cursor()

#drop table product
dropTable   = '''DROP TABLE product;'''
cursor.execute(dropTable)

#create table product
sql = '''CREATE TABLE product(id int auto_increment,supermarket varchar(255),\
nom_drive varchar(255),nom_driveUrl varchar(255),rayon_principal varchar(255),rayon_secondaire varchar(255),\
nom_produit varchar(255),prix_produit decimal(10, 2),prix_ratio decimal(10, 2),unite varchar(50), lien_image varchar(255),PRIMARY KEY (id));'''
cursor.execute(sql)


#import all product
dfList1 = pd.read_csv('/Users/minh/Desktop/Comparator/Scrap/Data_base/drivesScraped/list_drives_auchanScraped.csv')
dfList2 = pd.read_csv('/Users/minh/Desktop/Comparator/Scrap/Data_base/drivesScraped/list_drives_carrefourScraped.csv')
dfList3 = pd.read_csv('/Users/minh/Desktop/Comparator/Scrap/Data_base/drivesScraped/list_drives_casinoScraped.csv')

frames = [dfList1, dfList2, dfList3]

dfSupermarket= pd.concat(frames)
for i,supermarket in enumerate(dfSupermarket['supermarket']):
    department = dfSupermarket.iloc[i]["department"]
    city = dfSupermarket.iloc[i]["city"]
    dateScraped= dfSupermarket.iloc[i]["date_scraped"]
    nom_drive = dfSupermarket.iloc[i]["nom_drive"]

    path = f'/Users/minh/Desktop/Comparator/Scrap/Data_base/{department}/{city}/{nom_drive}/{dateScraped}/{supermarket}_produits.csv'
    
    with open(path, 'r') as f:
        reader = csv.DictReader(f, delimiter = ',')
        for row in reader:
            sql_statement = "INSERT INTO product(supermarket ,nom_drive,nom_driveUrl,rayon_principal ,rayon_secondaire,nom_produit,prix_produit,prix_ratio, unite,lien_image) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.executemany(sql_statement,[(row['supermarket'], row['nom_drive'], row['nom_driveUrl'], row['rayon_principal'], row['rayon_secondaire'], row['nom_produit'], row['prix_produit'], row['prix_ratio'], row['unite'], row['lien_image'])])
    print(f'import succes {nom_drive}')

conn.commit()
conn.close()

print("CSV data imported successfully into PostgreSQL.")


connect succes


KeyboardInterrupt: 